In [98]:
import os
import json
import pandas as pd
import folium
import gpxpy
import gpxpy.gpx
import requests
import math

If this value is true, the programme will fetch the data again from the ut.no API. Set this to false if you want to play around with the code.

Then the programme will get its data from a cached version in the data folder

In [99]:
fetch_data_from_api_again = True

### Helper functions

In [100]:
# Read and combine all json files
def read_json_files(folder_path):
    combined_edges = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                edges = data["data"]["ntb_findCabins"]["edges"]
                combined_edges.extend(edges)
    return combined_edges


In [101]:
# Convert data into pandas dataframe
def convert_to_dataframe(data):
    records = []
    for item in data:
        node = item["node"]
        geometry = node["geometry"]
        record = {
            "id": node["id"],
            "name": node["name"],
            "serviceLevel": node["serviceLevel"],
            "dntCabin": node["dntCabin"],
            "ownername": node["owner"]["name"],
            "latitude": geometry["coordinates"][1],  # longitude
            "longitude": geometry["coordinates"][0],  # latitude
            "height": geometry["coordinates"][2],  # height
            "areaName": node["areas"][0]["name"] if node["areas"] else "",  # area name
            "dntKey": node["openingHours"][0]["key"] if node["openingHours"] else "",
            "bedsStaffed": node["bedsStaffed"],
            "bedsNoService": node["bedsNoService"],
            "bedsSelfService": node["bedsSelfService"],
        }
        records.append(record)
    df = pd.DataFrame(records)
    return df


In [102]:
def getCounterTable(df: pd.DataFrame, column_name: str, sort_ascending=False):
	counts = df[column_name].value_counts()

	sorted_counts = counts.sort_values(ascending=sort_ascending)

	return pd.DataFrame({'Value': sorted_counts.index, 'Count': sorted_counts.values})

### Download from UT.no sin API

#### Static data
The API only allows the download of up to 500 huts at the same time. Therefore one needs to trigger the API several times.

At the moment, the number of huts is 645 (2024-03-15), so it will hardly be necessary to do more than 2 times in the future.

However, there is a static value being set to 500 and an automatism checking if there is more data later.

If this somehow does not work, try to change the static number, they might have changed the maximum

In [103]:
request_huts_per_request = 500
folder_path = './data/'

In [104]:
more_data_available = fetch_data_from_api_again
counter_pagination = 0
afterCursor = None

while more_data_available:
    post_data = {
        "operationName": "FindCabins",
        "variables": {
            "input": {
                "pageOptions": {
                    "limit": request_huts_per_request,
                    "afterCursor": afterCursor,
                    "orderByDirection": "DESC",
                    "orderBy": "ID",
                },
                "filters": {"and": [{"dntCabin": {"value": True}}]},
            }
        },
        "query": "query FindCabins($input: NTB_FindCabinsInput) {\n  ntb_findCabins(input: $input) {\n    totalCount\n    pageInfo {\n      hasNextPage\n      endCursor\n      __typename\n    }\n    edges {\n      node {\n        ...CabinFragment\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment CabinFragment on NTB_Cabin {\n  id\n  name\n  serviceLevel\n  bedsToday\n  bedsStaffed\n  bedsNoService\n  bedsSelfService\n  bedsWinter\n  dntCabin\n  owner {\n    name\n    __typename\n  }\n  accessibilities {\n    id\n    name\n    __typename\n  }\n  openingHours {\n    allYear\n    from\n    to\n    serviceLevel\n    key\n    __typename\n  }\n  geometry\n  media {\n    id\n    uri\n    type\n    description\n    tags\n    __typename\n  }\n  areas {\n    id\n    name\n    __typename\n  }\n  __typename\n}\n",
    }

    p_data = json.dumps(post_data)
    ua = 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0'

    # Make post request
    dnt_hytter_response = requests.post(
        "https://api.ut.no",
        data=p_data,
        headers={"Content-Type": "application/json", "User-Agent": ua},
    )

    if dnt_hytter_response.headers.get('content-type') == 'application/json':
        response_data = dnt_hytter_response.json()
    else:
        response_data = None

    
    page_info = response_data["data"]["ntb_findCabins"]["pageInfo"]

    json_obj = json.dumps(dnt_hytter_response.json(), indent=4)

    with open(f"{folder_path}hytter_{counter_pagination}.json", "w") as outfile:
        outfile.write(json_obj)

    if page_info["hasNextPage"]:
        counter_pagination += 1
        afterCursor = page_info["endCursor"]
    else:
        more_data_available = False

### Read json files

In [105]:
combined_data = read_json_files(folder_path)

### Convert to pandas

In [106]:
df = convert_to_dataframe(combined_data)

### Check length
Was 645 on 2024-03-15, should be "similar" at any later point

In [107]:
len(combined_data)

645

### Show table

In [108]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,areaName,dntKey,bedsStaffed,bedsNoService,bedsSelfService
0,101250515,Svartpoten,no-service,True,DNT Oslo og Omegn,59.919374,10.389953,355,Oslomarka,dnt-key,0,4,0
1,101250514,Svartvannskoia,no-service,True,DNT Oslo og Omegn,59.919473,10.390162,354,Oslomarka,dnt-key,0,3,0
2,101242071,Langhuken,no-service (no beds),True,Keipen Turlag,61.827808,5.444940,370,Fjordane,unlocked,0,0,0
3,101240142,Hallingskeid - Lokalet,self-service,True,Bergen og Hordaland Turlag,60.668296,7.248107,1095,Skarvheimen,None,0,0,23
4,101233402,Skjennungsvolden,no-service,True,DNT Oslo og Omegn,60.004078,10.682230,423,Oslomarka,dnt-key,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,1083,Damtjønna,emergency shelter,True,Trondhjems Turistforening,62.802600,9.891300,678,Trollheimen,unlocked,0,0,0
641,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland,dnt-key,0,28,0
642,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya",dnt-key,0,8,0
643,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli",unlocked,0,9,0


#### Prefilter
This applies filters to the table before anything else will happen. The standard code only has one filter.
It removes all huts where one cannot stay overnight, like emergency shelters

In [109]:
df = df[~df['serviceLevel'].isin(["emergency shelter", "food service", "no-service (no beds)", "closed"])]

In [110]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,areaName,dntKey,bedsStaffed,bedsNoService,bedsSelfService
0,101250515,Svartpoten,no-service,True,DNT Oslo og Omegn,59.919374,10.389953,355,Oslomarka,dnt-key,0,4,0
1,101250514,Svartvannskoia,no-service,True,DNT Oslo og Omegn,59.919473,10.390162,354,Oslomarka,dnt-key,0,3,0
3,101240142,Hallingskeid - Lokalet,self-service,True,Bergen og Hordaland Turlag,60.668296,7.248107,1095,Skarvheimen,None,0,0,23
4,101233402,Skjennungsvolden,no-service,True,DNT Oslo og Omegn,60.004078,10.682230,423,Oslomarka,dnt-key,0,7,0
5,101225968,Toralfbuda,no-service,True,DNT Sunnmøre,62.250761,5.875972,1,Sunnmøre,special key,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,1088,Hoemsbu,self-service,True,DNT Romsdal,62.565126,8.144124,37,Romsdalen og Eikesdalen,unlocked,0,0,31
641,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland,dnt-key,0,28,0
642,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya",dnt-key,0,8,0
643,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli",unlocked,0,9,0


In [111]:
df.sort_values(by=['areaName']).to_csv(f"{folder_path}alle_hytter.csv")

### Generate map

In [112]:
m = folium.Map(location=[60.472, 8.468], zoom_start=6)

# Add markers for each point in the DataFrame
for i, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        tooltip=f"Height: {row['height']} meters"
    ).add_to(m)

# Display the map
m.save(f"{folder_path}map_norway.html")  # Save the map to an HTML file

In [113]:
m

In [114]:
### Map Vestlandet (Norway most important region)

In [115]:
m_vest = folium.Map(location=[60.170, 6.971], zoom_start=8)
# Add markers for each point in the DataFrame
for i, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        tooltip=f"Height: {row['height']} meters"
    ).add_to(m_vest)

# Display the map
m_vest.save(f"{folder_path}map_vestlandet.html")  # Save the map to an HTML file

In [116]:
m_vest

### Export to GPX file

In [117]:
gpx = gpxpy.gpx.GPX()

# Create a GPX waypoint for each row in the DataFrame
for i, row in df.iterrows():
    waypoint = gpxpy.gpx.GPXWaypoint(latitude=row['latitude'], longitude=row['longitude'], elevation=row['height'])
    waypoint.name = row['name']

    gpx.waypoints.append(waypoint)

# Save the GPX to a file
with open(f"{folder_path}points.gpx", 'w') as f:
    f.write(gpx.to_xml())


### Statistics for nerds

In [118]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,areaName,dntKey,bedsStaffed,bedsNoService,bedsSelfService
0,101250515,Svartpoten,no-service,True,DNT Oslo og Omegn,59.919374,10.389953,355,Oslomarka,dnt-key,0,4,0
1,101250514,Svartvannskoia,no-service,True,DNT Oslo og Omegn,59.919473,10.390162,354,Oslomarka,dnt-key,0,3,0
3,101240142,Hallingskeid - Lokalet,self-service,True,Bergen og Hordaland Turlag,60.668296,7.248107,1095,Skarvheimen,None,0,0,23
4,101233402,Skjennungsvolden,no-service,True,DNT Oslo og Omegn,60.004078,10.682230,423,Oslomarka,dnt-key,0,7,0
5,101225968,Toralfbuda,no-service,True,DNT Sunnmøre,62.250761,5.875972,1,Sunnmøre,special key,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,1088,Hoemsbu,self-service,True,DNT Romsdal,62.565126,8.144124,37,Romsdalen og Eikesdalen,unlocked,0,0,31
641,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland,dnt-key,0,28,0
642,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya",dnt-key,0,8,0
643,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli",unlocked,0,9,0


##### Service level (betjent, selv-betjent, ubetjent)

In [119]:
getCounterTable(df, "serviceLevel")

,Value,Count
0,no-service,340
1,self-service,195
2,staffed,47


##### Region

In [120]:
getCounterTable(df, "areaName").head(20)

,Value,Count
0,Oslomarka,41
1,Ryfylke,28
2,"Stølsheimen, Bergsdalen og Vossafjelli",26
3,Hardangervidda,26
4,Breheimen med Jostedalsbreen,22
5,Hedmarken og Hedmarksvidda,18
6,Jotunheimen,17
7,"Lofoten, Vesterålen og Hinnøya",16
8,Saltfjellet og Svartisen,15
9,Skarvheimen,14


##### Owner (Is DNT or not)

In [121]:
getCounterTable(df, "dntCabin")

,Value,Count
0,True,582


In [122]:
getCounterTable(df, "ownername")

,Value,Count
0,DNT Oslo og Omegn,139
1,Stavanger Turistforening,45
2,Bergen og Hordaland Turlag,27
3,Kristiansund og Nordmøre Turistforening,26
4,DNT Drammen og Omegn,23
...,...,...
56,Notodden og Hjartdal Turlag,1
57,Ytre Sogn Turlag,1
58,Varangerhalvøya Turlag,1
59,Årdal Turlag,1


##### DNT-Nøkkel neaded

In [123]:
getCounterTable(df, "dntKey")

,Value,Count
0,dnt-key,287
1,unlocked,140
2,special key,81
3,,3
